In [0]:
#mount your blob storage account using SAS stored in AKV and Databricks secret scope
#see here on how to create Databricks secret scope: https://docs.microsoft.com/en-us/azure/databricks/security/secrets/secret-scopes

dbutils.fs.mount(
  source = "wasbs://<container>@<storageaccountname>.blob.core.windows.net/",
  mount_point = "/mnt/",
  extra_configs = {"fs.azure.sas.<container>.<storageaccountname>.blob.core.windows.net": dbutils.secrets.get(scope = "<databricks secret scope>", key = "<secret name in Key Vault>")}
)

In [0]:
import random
import datetime
import string
import csv

class record:
    def __init__(self, memberId, transactionDate, transactionId, sku, roundedAmountSpent):
        self.memberId = memberId
        self.transactionDate = transactionDate
        self.transactionId = transactionId
        self.sku = sku
        self.roundedAmountSpent = roundedAmountSpent

# How many lines to generate
numberofrecords = 100

# Maximum number of line items per transaction
maxlineitems = 5

# lists of Member IDs and Skus
memberIds = ['1320122100','7611334511','8549494944','5454667577','6523454446','4335353354','5657677755','4648889113','5484873335','6543554443','5325553353','5325553303','5325553323','5325553313']
skus = ['UY0122100','OO1334511','8549494DX', 'DUS872344']

startdate, enddate = datetime.date(2022, 1, 1), datetime.date(2022, 12, 31)
datelist = [startdate]

while startdate != enddate:
    startdate += datetime.timedelta(days=1)
    datelist.append(startdate)

# Starting transaction ID
tranid = 100

recordlist = []

while (len(recordlist) <= numberofrecords):
    # make records
    memberId = random.choice(memberIds)
    transactionDate = random.choice(datelist)

    #Transaction ID Format
	#changed rjust(4) to rjust(3)
    transactionId = 'TRAN' + str(tranid).rjust(3, '0') + 'ABC'
    lineitems = random.randint(1, maxlineitems)
    linecount = 1
    while(linecount <= lineitems and len(recordlist) <= numberofrecords):
        sku = random.choice(skus)

        #Amount spend min, max
        roundedAmountSpent = random.randint(5, 50)
        recordlist.append([memberId, transactionDate, transactionId, str(linecount), sku, roundedAmountSpent])
        linecount += 1
    tranid += 1

#File the csv will be written to mounted blob storage account container with the filename you input below
with open('/dbfs/mnt/<input filename for data ending in .csv>', 'w') as csv_file:
	wr = csv.writer(csv_file, delimiter=',')
	wr.writerow(["memberId", "transactionDate", "transactionId", "lineitems", "sku", "roundedAmountSpent"])
	wr.writerows(recordlist)